An implementation of sequence to sequence learning for performing multiplication. Would it work?

Input: "99*101"
Output: "9999"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be reversed, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

In [1]:
from __future__ import print_function
import numpy as np
from six.moves import range
import sys

In [28]:
import keras as K
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.regularizers import L1L2

# config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
config = tf.ConfigProto(device_count = {'GPU': 1 , 'CPU': 8} ) 
config.gpu_options.allow_growth = True

sess = tf.Session(config=config) 
K.backend.set_session(sess)

In [3]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.

        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.

        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [4]:
# All the numbers, plus sign and space for padding.
chars = '0123456789x '
ctable = CharacterTable(chars)

In [5]:
ctable.encode(C='0 x', num_rows=10)

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [6]:
x = ctable.encode(C='0 x', num_rows=5)
ctable.decode(x)

'0 x  '

In [7]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [8]:
def generate_training(TRAINING_SIZE = 50000, DIGITS = 3, REVERSE = True):

    # Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
    # int is DIGITS.
    MAXLEN = DIGITS + 1 + DIGITS

    questions = []
    answers = []
    seen = set()
    print('Generating data...')
    while len(questions) < TRAINING_SIZE:
        f = lambda: int(''.join(np.random.choice(list('0123456789'))
                        for i in range(np.random.randint(1, DIGITS + 1))))
        a, b = f(), f()
        # Skip any addition questions we've already seen
        # Also skip any such that x+Y == Y+x (hence the sorting).
        key = tuple(sorted((a, b)))
        if key in seen:
            continue
        seen.add(key)
        # Pad the data with spaces such that it is always MAXLEN.
        q = '{}x{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a * b)
        # Answers can be of maximum size 2*DIGITS.
        ans += ' ' * (2*DIGITS - len(ans))
        if REVERSE:
            # Reverse the query, e.g., '12*345  ' becomes '  543*21'. (Note the
            # space used for padding.)
            query = query[::-1]
        questions.append(query)
        answers.append(ans)
    print('Total addition questions:', len(questions))
    out = np.vstack((np.array(questions), np.array(answers)))
    return out.T

In [9]:
train2char = generate_training(TRAINING_SIZE=5000, DIGITS=2, REVERSE=True)

Generating data...
Total addition questions: 5000


In [10]:
train2char.shape

(5000, 2)

In [11]:
r = np.random.randint(low=0, high=train2char.shape[0], size=4)
train2char[r, :]

array([['86x06', '4080'],
       ['14x27', '2952'],
       ['32x36', '1449'],
       ['71x49', '1598']], dtype='<U5')

In [12]:
def vectorization(train, DIGITS):
    MAXLEN_IN = 2*DIGITS+1
    MAXLEN_OUT = 2*DIGITS
    print('Vectorization...')
    x = np.zeros((train.shape[0], MAXLEN_IN, len(chars)), dtype=np.bool)
    y = np.zeros((train.shape[0], MAXLEN_OUT, len(chars)), dtype=np.bool)
    # encode questions
    for i, sentence in enumerate(train[:, 0]):
        x[i] = ctable.encode(sentence, MAXLEN_IN)
    # encode answers
    for i, sentence in enumerate(train[:, 1]):
        y[i] = ctable.encode(sentence, MAXLEN_OUT)

    # Shuffle (x, y) in unison as the later parts of x will almost all be larger
    # digits.
    indices = np.arange(len(y))
    np.random.shuffle(indices)
    x = x[indices]
    y = y[indices]

    # Explicitly set apart 10% for validation data that we never train over.
    split_at = len(x) - len(x) // 10
    (x_train, x_val) = x[:split_at], x[split_at:]
    (y_train, y_val) = y[:split_at], y[split_at:]

    print('Shapes in training Data:')
    print(x_train.shape)
    print(y_train.shape)

    print('Shapes in validation Data:')
    print(x_val.shape)
    print(y_val.shape)
    return x_train, y_train, x_val, y_val

In [13]:
x_train_2char, y_train_2char, x_val_2char, y_val_2char = vectorization(train=train2char, DIGITS=2)

Vectorization...
Shapes in training Data:
(4500, 5, 12)
(4500, 4, 12)
Shapes in validation Data:
(500, 5, 12)
(500, 4, 12)


In [27]:
layers.GRU??

In [29]:
def build_model(chars, rnn_type='gru', DIGITS=3, HIDDEN_SIZE=10, BATCH_SIZE=100, DECODER_LAYERS=1, REG=L1L2(0.0,0.0)):
    MAXLEN_IN = 2*DIGITS+1
    MAXLEN_OUT = 2*DIGITS
    if rnn_type.lower() == 'gru':
        RNN = layers.GRU
    elif rnn_type.lower() == 'lstm':
        RNN = layers.LSTM
    elif rnn_type.lower() == 'rnn':
        RNN = layers.SimpleRNN    
    else:
        print('{rnn_type} RNN type not covered'.format(rnn_type=rnn_type))
        sys.exit(0)
        
    print('Build model...')
    model = Sequential()
    # "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
    # Note: In a situation where your input sequences have a variable length,
    # use input_shape=(None, num_feature).
    model.add(RNN(units=HIDDEN_SIZE, input_shape=(MAXLEN_IN, len(chars)), recurrent_regularizer=REG))
    # As the decoder RNN's input, repeatedly provide with the last output of
    # RNN for each time step. 
    model.add(layers.RepeatVector(MAXLEN_OUT))
    # The decoder RNN could be multiple layers stacked or a single layer.
    for _ in range(DECODER_LAYERS):
        # By setting return_sequences to True, return not only the last output but
        # all the outputs so far in the form of (num_samples, timesteps,
        # output_dim). This is necessary as TimeDistributed in the below expects
        # the first dimension to be the timesteps.
        model.add(RNN(HIDDEN_SIZE, return_sequences=True, recurrent_regularizer=REG))

    # Apply a dense layer to the every temporal slice of an input. For each of step
    # of the output sequence, decide which character should be chosen.
    model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()
    return model

In [39]:
model2char = build_model(chars=chars, rnn_type='lstm', DIGITS=2, HIDDEN_SIZE=64, BATCH_SIZE=128, DECODER_LAYERS=1, REG=L1L2(0.05,0.05))

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 64)                19712     
_________________________________________________________________
repeat_vector_8 (RepeatVecto (None, 4, 64)             0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 4, 64)             33024     
_________________________________________________________________
time_distributed_8 (TimeDist (None, 4, 12)             780       
Total params: 53,516
Trainable params: 53,516
Non-trainable params: 0
_________________________________________________________________


In [40]:
def train(model, x_train, y_train, x_val, y_val,  n_epochs=20, BATCH_SIZE=128, REVERSE=False):
    # Train the model each generation and show predictions against the validation
    # dataset.
    for iteration in range(1, n_epochs+1):
        print()
        print('-' * 50)
        print('Iteration', iteration)
        model.fit(x_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=1,
                  validation_data=(x_val, y_val), shuffle=False)
        model.reset_states()
        # Select 10 samples from the validation set at random so we can visualize
        # errors.
        for i in range(5):
            ind = np.random.randint(0, len(x_val))
            rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
            preds = model.predict_classes(rowx, verbose=0)
            q = ctable.decode(rowx[0])
            correct = ctable.decode(rowy[0])
            guess = ctable.decode(preds[0], calc_argmax=False)
            print('Question: ', q[::-1] if REVERSE else q, end=' ')
            print('Answer: ', correct, end=' ')
            if correct == guess:
                print(colors.ok + '☑' + colors.close, end=' ')
            else:
                print(colors.fail + '☒' + colors.close, end=' ')
            print('Guess: ', guess, end='\n')

In [41]:
train(model2char, x_train_2char, y_train_2char, x_val_2char, y_val_2char,  n_epochs=500, 
      BATCH_SIZE=128, REVERSE=True)


--------------------------------------------------
Iteration 1
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 2s 522us/step - loss: 65.5966 - acc: 0.1289 - val_loss: 42.0265 - val_acc: 0.1190
Question:  63x13 Answer:  819  ☒ Guess:      
Question:  13x1  Answer:  13   ☒ Guess:      
Question:  40x9  Answer:  360  ☒ Guess:      
Question:  29x61 Answer:  1769 ☒ Guess:      
Question:  67x67 Answer:  4489 ☒ Guess:      

--------------------------------------------------
Iteration 2
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0s 108us/step - loss: 28.4440 - acc: 0.1313 - val_loss: 16.9838 - val_acc: 0.1600
Question:  70x55 Answer:  3850 ☒ Guess:  0000
Question:  91x69 Answer:  6279 ☒ Guess:      
Question:  39x16 Answer:  624  ☒ Guess:      
Question:  9x32  Answer:  288  ☒ Guess:      
Question:  26x20 Answer:  520  ☒ Guess:      

-------------------------------------------

4500/4500 [==============================] - 0s 102us/step - loss: 2.1539 - acc: 0.2806 - val_loss: 2.1468 - val_acc: 0.2820
Question:  82x62 Answer:  5084 ☒ Guess:  1266
Question:  93x89 Answer:  8277 ☒ Guess:  1166
Question:  38x63 Answer:  2394 ☒ Guess:  1226
Question:  6x58  Answer:  348  ☒ Guess:  10  
Question:  24x65 Answer:  1560 ☒ Guess:  1200

--------------------------------------------------
Iteration 18
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0s 100us/step - loss: 2.1509 - acc: 0.2823 - val_loss: 2.1425 - val_acc: 0.2825
Question:  95x6  Answer:  570  ☒ Guess:  150 
Question:  56x43 Answer:  2408 ☒ Guess:  1222
Question:  97x56 Answer:  5432 ☒ Guess:  1550
Question:  62x26 Answer:  1612 ☒ Guess:  1228
Question:  95x6  Answer:  570  ☒ Guess:  150 

--------------------------------------------------
Iteration 19
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0

4500/4500 [==============================] - 0s 101us/step - loss: 2.1158 - acc: 0.2908 - val_loss: 2.1114 - val_acc: 0.2985
Question:  87x50 Answer:  4350 ☒ Guess:  1000
Question:  94x51 Answer:  4794 ☒ Guess:  1500
Question:  80x55 Answer:  4400 ☒ Guess:  5000
Question:  66x78 Answer:  5148 ☒ Guess:  1446
Question:  27x76 Answer:  2052 ☒ Guess:  1166

--------------------------------------------------
Iteration 34
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0s 99us/step - loss: 2.1142 - acc: 0.2912 - val_loss: 2.1092 - val_acc: 0.2990
Question:  8x60  Answer:  480  ☒ Guess:  20  
Question:  70x72 Answer:  5040 ☒ Guess:  1000
Question:  64x9  Answer:  576  ☒ Guess:  12  
Question:  89x24 Answer:  2136 ☒ Guess:  1266
Question:  9x59  Answer:  531  ☒ Guess:  15  

--------------------------------------------------
Iteration 35
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0s

Epoch 1/1
4500/4500 [==============================] - 0s 103us/step - loss: 2.0890 - acc: 0.2996 - val_loss: 2.0869 - val_acc: 0.3080
Question:  45x59 Answer:  2655 ☒ Guess:  1555
Question:  63x46 Answer:  2898 ☒ Guess:  1224
Question:  19x88 Answer:  1672 ☒ Guess:  1666
Question:  96x24 Answer:  2304 ☒ Guess:  2266
Question:  77x20 Answer:  1540 ☒ Guess:  1000

--------------------------------------------------
Iteration 50
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0s 99us/step - loss: 2.0878 - acc: 0.2999 - val_loss: 2.0870 - val_acc: 0.3080
Question:  46x26 Answer:  1196 ☒ Guess:  1228
Question:  52x82 Answer:  4264 ☒ Guess:  1244
Question:  70x45 Answer:  3150 ☒ Guess:  2000
Question:  95x89 Answer:  8455 ☒ Guess:  3550
Question:  31x88 Answer:  2728 ☒ Guess:  1266

--------------------------------------------------
Iteration 51
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==========================

4500/4500 [==============================] - 0s 102us/step - loss: 2.0666 - acc: 0.3132 - val_loss: 2.0668 - val_acc: 0.3125
Question:  3x1   Answer:  3    ☒ Guess:  1   
Question:  52x82 Answer:  4264 ☒ Guess:  1244
Question:  6x19  Answer:  114  ☒ Guess:  16  
Question:  36x12 Answer:  432  ☒ Guess:  128 
Question:  72x20 Answer:  1440 ☒ Guess:  1000

--------------------------------------------------
Iteration 66
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0s 99us/step - loss: 2.0653 - acc: 0.3138 - val_loss: 2.0673 - val_acc: 0.3115
Question:  90x92 Answer:  8280 ☒ Guess:  1000
Question:  63x46 Answer:  2898 ☒ Guess:  1224
Question:  96x24 Answer:  2304 ☒ Guess:  1668
Question:  2x17  Answer:  34   ☒ Guess:  12  
Question:  53x8  Answer:  424  ☒ Guess:  125 

--------------------------------------------------
Iteration 67
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0s

...

4500/4500 [==============================] - 0s 101us/step - loss: 1.6426 - acc: 0.4936 - val_loss: 1.7684 - val_acc: 0.4425
Question:  10x6  Answer:  60   ☒ Guess:  00  
Question:  95x15 Answer:  1425 ☒ Guess:  1255
Question:  50x84 Answer:  4200 ☒ Guess:  2200
Question:  49x6  Answer:  294  ☒ Guess:  344 
Question:  63x92 Answer:  5796 ☒ Guess:  2666

--------------------------------------------------
Iteration 434
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0s 103us/step - loss: 1.6415 - acc: 0.4948 - val_loss: 1.7682 - val_acc: 0.4420
Question:  18x74 Answer:  1332 ☒ Guess:  1122
Question:  35x95 Answer:  3325 ☒ Guess:  2255
Question:  71x84 Answer:  5964 ☒ Guess:  3522
Question:  12x23 Answer:  276  ☒ Guess:  666 
Question:  81x53 Answer:  4293 ☒ Guess:  2333

--------------------------------------------------
Iteration 435
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] -

4500/4500 [==============================] - 0s 95us/step - loss: 1.6315 - acc: 0.4987 - val_loss: 1.7634 - val_acc: 0.4440
Question:  15x1  Answer:  15   ☒ Guess:  55  
Question:  20x23 Answer:  460  ☒ Guess:  660 
Question:  74x28 Answer:  2072 ☒ Guess:  2222
Question:  8x20  Answer:  160  ☑ Guess:  160 
Question:  3x26  Answer:  78   ☒ Guess:  12  

--------------------------------------------------
Iteration 450
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0s 95us/step - loss: 1.6300 - acc: 0.4991 - val_loss: 1.7624 - val_acc: 0.4450
Question:  21x32 Answer:  672  ☒ Guess:  232 
Question:  63x13 Answer:  819  ☒ Guess:  199 
Question:  87x50 Answer:  4350 ☒ Guess:  4500
Question:  77x51 Answer:  3927 ☒ Guess:  4777
Question:  89x88 Answer:  7832 ☒ Guess:  7722

--------------------------------------------------
Iteration 451
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0

4500/4500 [==============================] - 0s 95us/step - loss: 1.6191 - acc: 0.5029 - val_loss: 1.7589 - val_acc: 0.4455
Question:  52x11 Answer:  572  ☒ Guess:  122 
Question:  2x0   Answer:  0    ☑ Guess:  0   
Question:  19x25 Answer:  475  ☒ Guess:  155 
Question:  15x13 Answer:  195  ☒ Guess:  255 
Question:  76x51 Answer:  3876 ☒ Guess:  4466

--------------------------------------------------
Iteration 466
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0s 98us/step - loss: 1.6181 - acc: 0.5039 - val_loss: 1.7587 - val_acc: 0.4465
Question:  30x7  Answer:  210  ☒ Guess:  10  
Question:  45x96 Answer:  4320 ☒ Guess:  2400
Question:  94x51 Answer:  4794 ☒ Guess:  4942
Question:  83x84 Answer:  6972 ☒ Guess:  3744
Question:  18x2  Answer:  36   ☒ Guess:  26  

--------------------------------------------------
Iteration 467
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0

4500/4500 [==============================] - 0s 99us/step - loss: 1.6076 - acc: 0.5072 - val_loss: 1.7534 - val_acc: 0.4470
Question:  33x19 Answer:  627  ☒ Guess:  107 
Question:  29x61 Answer:  1769 ☒ Guess:  1199
Question:  33x44 Answer:  1452 ☒ Guess:  1122
Question:  85x27 Answer:  2295 ☒ Guess:  2255
Question:  45x14 Answer:  630  ☒ Guess:  200 

--------------------------------------------------
Iteration 482
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0s 99us/step - loss: 1.6067 - acc: 0.5075 - val_loss: 1.7541 - val_acc: 0.4475
Question:  58x44 Answer:  2552 ☒ Guess:  2222
Question:  22x0  Answer:  0    ☑ Guess:  0   
Question:  97x58 Answer:  5626 ☒ Guess:  6662
Question:  26x85 Answer:  2210 ☒ Guess:  1100
Question:  45x47 Answer:  2115 ☒ Guess:  2225

--------------------------------------------------
Iteration 483
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0

4500/4500 [==============================] - 0s 101us/step - loss: 1.5970 - acc: 0.5106 - val_loss: 1.7496 - val_acc: 0.4515
Question:  88x30 Answer:  2640 ☒ Guess:  2440
Question:  37x41 Answer:  1517 ☒ Guess:  1777
Question:  10x33 Answer:  330  ☑ Guess:  330 
Question:  1x99  Answer:  99   ☑ Guess:  99  
Question:  8x61  Answer:  488  ☒ Guess:  46  

--------------------------------------------------
Iteration 498
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] - 0s 100us/step - loss: 1.5962 - acc: 0.5108 - val_loss: 1.7503 - val_acc: 0.4525
Question:  45x96 Answer:  4320 ☒ Guess:  2400
Question:  11x11 Answer:  121  ☒ Guess:  111 
Question:  55x96 Answer:  5280 ☒ Guess:  4000
Question:  27x49 Answer:  1323 ☒ Guess:  1113
Question:  84x53 Answer:  4452 ☒ Guess:  3366

--------------------------------------------------
Iteration 499
Train on 4500 samples, validate on 500 samples
Epoch 1/1
4500/4500 [==============================] -

## Try with more digits

In [42]:
train3char = generate_training(TRAINING_SIZE=400000, DIGITS=3, REVERSE=True)

Generating data...
Total addition questions: 400000


In [43]:
r = np.random.randint(low=0, high=train3char.shape[0], size=4)
train3char[r, :]

array([[' 57x968', '65175 '],
       ['389x354', '445299'],
       ['191x926', '120139'],
       [' 93x077', '30030 ']], dtype='<U7')

In [44]:
x_train_3char, y_train_3char, x_val_3char, y_val_3char = vectorization(train=train3char, DIGITS=3)

Vectorization...
Shapes in training Data:
(360000, 7, 12)
(360000, 6, 12)
Shapes in validation Data:
(40000, 7, 12)
(40000, 6, 12)


In [45]:
model3char = build_model(chars=chars, rnn_type='lstm', DIGITS=3, HIDDEN_SIZE=128, BATCH_SIZE=128, DECODER_LAYERS=1, REG=L1L2(0.0,0.05))

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 128)               72192     
_________________________________________________________________
repeat_vector_9 (RepeatVecto (None, 6, 128)            0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 6, 128)            131584    
_________________________________________________________________
time_distributed_9 (TimeDist (None, 6, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [47]:
train(model3char, x_train_3char, y_train_3char, x_val_3char, y_val_3char,  n_epochs=50, 
      BATCH_SIZE=128, REVERSE=True)


--------------------------------------------------
Iteration 1
Train on 360000 samples, validate on 40000 samples
Epoch 1/1
360000/360000 [==============================] - 45s 124us/step - loss: 2.0781 - acc: 0.2293 - val_loss: 2.0308 - val_acc: 0.2428
Question:  183x868 Answer:  158844 ☒ Guess:  118888
Question:  44x307  Answer:  13508  ☒ Guess:  1300  
Question:  32x336  Answer:  10752  ☒ Guess:  1188  
Question:  238x837 Answer:  199206 ☒ Guess:  113888
Question:  429x883 Answer:  378807 ☒ Guess:  133888

--------------------------------------------------
Iteration 2
Train on 360000 samples, validate on 40000 samples
Epoch 1/1
360000/360000 [==============================] - 43s 120us/step - loss: 2.0004 - acc: 0.2544 - val_loss: 1.9738 - val_acc: 0.2619
Question:  374x629 Answer:  235246 ☒ Guess:  213888
Question:  96x456  Answer:  43776  ☒ Guess:  4348  
Question:  508x245 Answer:  124460 ☒ Guess:  133200
Question:  334x134 Answer:  44756  ☒ Guess:  11888 
Question:  463x796 Ans

360000/360000 [==============================] - 44s 121us/step - loss: 1.5209 - acc: 0.4806 - val_loss: 1.5087 - val_acc: 0.4840
Question:  369x377 Answer:  139113 ☒ Guess:  144333
Question:  446x829 Answer:  369734 ☒ Guess:  388844
Question:  873x3   Answer:  2619   ☒ Guess:  2899  
Question:  255x717 Answer:  182835 ☒ Guess:  144005
Question:  173x444 Answer:  76812  ☒ Guess:  66112 

--------------------------------------------------
Iteration 17
Train on 360000 samples, validate on 40000 samples
Epoch 1/1
360000/360000 [==============================] - 45s 125us/step - loss: 1.5099 - acc: 0.4834 - val_loss: 1.4990 - val_acc: 0.4861
Question:  427x839 Answer:  358253 ☒ Guess:  394433
Question:  383x436 Answer:  166988 ☒ Guess:  114488
Question:  326x839 Answer:  273514 ☒ Guess:  218844
Question:  416x575 Answer:  239200 ☒ Guess:  238900
Question:  27x550  Answer:  14850  ☒ Guess:  14550 

--------------------------------------------------
Iteration 18
Train on 360000 samples, vali

360000/360000 [==============================] - 45s 126us/step - loss: 1.4252 - acc: 0.5112 - val_loss: 1.4302 - val_acc: 0.5113
Question:  160x26  Answer:  4160   ☒ Guess:  3360  
Question:  26x462  Answer:  12012  ☒ Guess:  11122 
Question:  393x575 Answer:  225975 ☒ Guess:  299915
Question:  849x94  Answer:  79806  ☒ Guess:  73666 
Question:  299x651 Answer:  194649 ☒ Guess:  178849

--------------------------------------------------
Iteration 32
Train on 360000 samples, validate on 40000 samples
Epoch 1/1
360000/360000 [==============================] - 45s 125us/step - loss: 1.4215 - acc: 0.5124 - val_loss: 1.4267 - val_acc: 0.5128
Question:  341x432 Answer:  147312 ☒ Guess:  143172
Question:  258x391 Answer:  100878 ☒ Guess:  110988
Question:  86x175  Answer:  15050  ☒ Guess:  11400 
Question:  549x685 Answer:  376065 ☒ Guess:  488115
Question:  185x457 Answer:  84545  ☒ Guess:  89995 

--------------------------------------------------
Iteration 33
Train on 360000 samples, vali

360000/360000 [==============================] - 45s 124us/step - loss: 1.3841 - acc: 0.5265 - val_loss: 1.3975 - val_acc: 0.5250
Question:  290x494 Answer:  143260 ☒ Guess:  148660
Question:  331x61  Answer:  20191  ☒ Guess:  11111 
Question:  556x237 Answer:  131772 ☒ Guess:  133192
Question:  128x377 Answer:  48256  ☒ Guess:  44416 
Question:  396x663 Answer:  262548 ☒ Guess:  244488

--------------------------------------------------
Iteration 47
Train on 360000 samples, validate on 40000 samples
Epoch 1/1
360000/360000 [==============================] - 45s 124us/step - loss: 1.3821 - acc: 0.5272 - val_loss: 1.3962 - val_acc: 0.5252
Question:  967x295 Answer:  285265 ☒ Guess:  266825
Question:  14x662  Answer:  9268   ☒ Guess:  8008  
Question:  895x389 Answer:  348155 ☒ Guess:  331115
Question:  541x139 Answer:  75199  ☒ Guess:  77999 
Question:  109x21  Answer:  2289   ☒ Guess:  2299  

--------------------------------------------------
Iteration 48
Train on 360000 samples, vali

In [48]:
model3charbis = build_model(chars=chars, rnn_type='lstm', DIGITS=3, HIDDEN_SIZE=256, 
                         BATCH_SIZE=128, DECODER_LAYERS=1, REG=L1L2(0.0,0.05))

train(model3charbis, x_train_3char, y_train_3char, x_val_3char, y_val_3char,  n_epochs=50, 
      BATCH_SIZE=128, REVERSE=True)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 256)               275456    
_________________________________________________________________
repeat_vector_10 (RepeatVect (None, 6, 256)            0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 6, 256)            525312    
_________________________________________________________________
time_distributed_10 (TimeDis (None, 6, 12)             3084      
Total params: 803,852
Trainable params: 803,852
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 1
Train on 360000 samples, validate on 40000 samples
Epoch 1/1
360000/360000 [==============================] - 49s 136us/step - loss: 2.2673 - acc: 0.2272 - val_loss: 2.0287 - val_acc: 0.246

360000/360000 [==============================] - 47s 130us/step - loss: 1.4371 - acc: 0.5109 - val_loss: 1.4372 - val_acc: 0.5112
Question:  994x713 Answer:  708722 ☒ Guess:  794422
Question:  192x404 Answer:  77568  ☒ Guess:  77988 
Question:  644x149 Answer:  95956  ☒ Guess:  91136 
Question:  535x868 Answer:  464380 ☒ Guess:  446680
Question:  621x187 Answer:  116127 ☒ Guess:  110077

--------------------------------------------------
Iteration 15
Train on 360000 samples, validate on 40000 samples
Epoch 1/1
360000/360000 [==============================] - 47s 131us/step - loss: 1.4271 - acc: 0.5144 - val_loss: 1.4275 - val_acc: 0.5140
Question:  891x299 Answer:  266409 ☒ Guess:  236699
Question:  105x820 Answer:  86100  ☒ Guess:  80100 
Question:  867x823 Answer:  713541 ☒ Guess:  710011
Question:  762x892 Answer:  679704 ☒ Guess:  666544
Question:  794x466 Answer:  370004 ☒ Guess:  342224

--------------------------------------------------
Iteration 16
Train on 360000 samples, vali

360000/360000 [==============================] - 46s 129us/step - loss: 1.3428 - acc: 0.5471 - val_loss: 1.3600 - val_acc: 0.5374
Question:  824x509 Answer:  419416 ☒ Guess:  449966
Question:  740x611 Answer:  452140 ☒ Guess:  495940
Question:  545x831 Answer:  452895 ☒ Guess:  449445
Question:  533x164 Answer:  87412  ☒ Guess:  88912 
Question:  953x675 Answer:  643275 ☒ Guess:  665555

--------------------------------------------------
Iteration 30
Train on 360000 samples, validate on 40000 samples
Epoch 1/1
360000/360000 [==============================] - 46s 129us/step - loss: 1.3385 - acc: 0.5488 - val_loss: 1.3560 - val_acc: 0.5384
Question:  603x476 Answer:  287028 ☒ Guess:  278088
Question:  184x801 Answer:  147384 ☒ Guess:  144444
Question:  300x572 Answer:  171600 ☒ Guess:  166600
Question:  583x869 Answer:  506627 ☒ Guess:  588007
Question:  405x895 Answer:  362475 ☒ Guess:  399425

--------------------------------------------------
Iteration 31
Train on 360000 samples, vali

360000/360000 [==============================] - 47s 129us/step - loss: 1.2933 - acc: 0.5666 - val_loss: 1.3178 - val_acc: 0.5534
Question:  42x471  Answer:  19782  ☒ Guess:  19922 
Question:  223x448 Answer:  99904  ☒ Guess:  90114 
Question:  72x824  Answer:  59328  ☒ Guess:  50088 
Question:  906x171 Answer:  154926 ☒ Guess:  166666
Question:  483x683 Answer:  329889 ☒ Guess:  382299

--------------------------------------------------
Iteration 45
Train on 360000 samples, validate on 40000 samples
Epoch 1/1
360000/360000 [==============================] - 46s 129us/step - loss: 1.2906 - acc: 0.5676 - val_loss: 1.3164 - val_acc: 0.5543
Question:  700x458 Answer:  320600 ☒ Guess:  306600
Question:  452x928 Answer:  419456 ☒ Guess:  444996
Question:  314x11  Answer:  3454   ☒ Guess:  3314  
Question:  897x994 Answer:  891618 ☒ Guess:  888138
Question:  66x49   Answer:  3234   ☒ Guess:  3134  

--------------------------------------------------
Iteration 46
Train on 360000 samples, vali